# ReAct agent with tool calling

This notebook walks through an example creating a ReAct Agent that uses tool calling.
This is useful for getting started quickly.
However, it is highly likely you will want to customize the logic - for information on that, check out the other examples in this folder.

## Set up the chat model and tools

Here we will define the chat model and tools that we want to use.
Importantly, this model MUST support OpenAI function calling.

In [1]:
%pip install --quiet -U langgraph langchain langchain_openai tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [2]:
%pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.4 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent

In [5]:
tools = [TavilySearchResults(max_results=1)]
model = ChatOpenAI()

## Create executor

We can now use the high level interface to create the executor

In [6]:
app = create_react_agent(model, tools=tools)

We can now invoke this executor. The input to this must be a dictionary with a single `messages` key that contains a list of messages.

In [7]:
inputs = {"messages": [HumanMessage(content="what is the weather in San Jerónimo de Tunán and Tarma, Junin, Perú?")]}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e8AdRqfb2twdUOBsEFhafIq8', 'function': {'arguments': '{"query": "weather in San Jerónimo de Tunán, Junin, Peru"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_IgOJN1PsIn3u1EISW0zuSrVo', 'function': {'arguments': '{"query": "weather in Tarma, Junin, Peru"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 103, 'total_tokens': 175}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f3c5e2d6-65b7-40d3-943b-13f1ceeb7de7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Jerónimo de Tunán, Junin, Peru'}, 'id': 'call_e8AdRqfb2twdUOBsEFhafIq8', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Tarma, Junin, Peru'}, 'id': 'call_IgOJN1PsIn3u1EISW0